# Dask DataFrame

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы: 
* Макрушин С.В. Лекция "Dask DataFrame"
* https://docs.dask.org/en/latest/dataframe.html
* Jesse C. Daniel. Data Science with Python and Dask. 

* https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.memory_usage_per_partition.html#dask.dataframe.DataFrame.memory_usage_per_partition
* https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.map_partitions.html#dask.dataframe.DataFrame.map_partitions
* https://docs.dask.org/en/stable/generated/dask.array.histogram.html
* https://docs.dask.org/en/stable/dataframe-categoricals.html
* https://docs.dask.org/en/stable/generated/dask.dataframe.DataFrame.pivot_table.html#dask.dataframe.DataFrame.pivot_table
* https://docs.dask.org/en/stable/generated/dask.dataframe.groupby.DataFrameGroupBy.apply.html
* https://docs.dask.org/en/stable/generated/dask.dataframe.Series.map_overlap.html#dask.dataframe.Series.map_overlap

* https://docs.dask.org/en/stable/best-practices.html
* https://docs.dask.org/en/stable/dashboard.html
* https://distributed.dask.org/en/stable/client.html

## Задачи для совместного разбора

In [1]:
import dask

In [2]:
!pip install --user distributed==2022.11.1
!pip install --user bokeh==2.4.2

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


1\. Создать `dask.DataFrame`. Рассмотреть основные возможности (выбор строк, группировки, работа с датами, добавление столбцов)

In [2]:
df = dask.datasets.timeseries(start='2000-01-01', end='2003-10-10')
df

,name,id,x,y
npartitions=1378,,,,
2000-01-01,object,int32,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2003-10-09,...,...,...,...
2003-10-10,...,...,...,...


In [3]:
df['xy'] = df['x'] + df['y']
df.head(5)

,name,id,x,y,xy
timestamp,,,,,
2000-01-01 00:00:00,Frank,969,-0.469833,0.005958,-0.463875
2000-01-01 00:00:01,Oliver,1008,0.924752,-0.562681,0.362071
2000-01-01 00:00:02,George,1037,-0.031205,0.035167,0.003962
2000-01-01 00:00:03,Wendy,995,0.126318,-0.867870,-0.741552
2000-01-01 00:00:04,Jerry,962,-0.086203,-0.697568,-0.783771


In [4]:
df[df['xy'] >= df['x'].mean()].head(5)

,name,id,x,y,xy
timestamp,,,,,
2000-01-01 00:00:01,Oliver,1008,0.924752,-0.562681,0.362071
2000-01-01 00:00:02,George,1037,-0.031205,0.035167,0.003962
2000-01-01 00:00:06,Dan,976,-0.240677,0.451280,0.210603
2000-01-01 00:00:10,Michael,954,-0.034515,0.300603,0.266088
2000-01-01 00:00:11,Alice,1007,0.467424,0.921368,1.388791


In [5]:
%%time
df.groupby(df.index.dt.to_period('M'))['x'].size().compute()

Wall time: 27.2 s


timestamp
2000-01    2678400
2000-02    2505600
2000-03    2678400
2000-04    2592000
2000-05    2678400
2000-06    2592000
2000-07    2678400
2000-08    2678400
2000-09    2592000
2000-10    2678400
2000-11    2592000
2000-12    2678400
2001-01    2678400
2001-02    2419200
2001-03    2678400
2001-04    2592000
2001-05    2678400
2001-06    2592000
2001-07    2678400
2001-08    2678400
2001-09    2592000
2001-10    2678400
2001-11    2592000
2001-12    2678400
2002-01    2678400
2002-02    2419200
2002-03    2678400
2002-04    2592000
2002-05    2678400
2002-06    2592000
2002-07    2678400
2002-08    2678400
2002-09    2592000
2002-10    2678400
2002-11    2592000
2002-12    2678400
2003-01    2678400
2003-02    2419200
2003-03    2678400
2003-04    2592000
2003-05    2678400
2003-06    2592000
2003-07    2678400
2003-08    2678400
2003-09    2592000
2003-10     777600
Freq: M, Name: x, dtype: int64

2\. Рассмотреть средства диагностики `dask`

In [10]:
from dask.diagnostics import ProgressBar

In [12]:
%%time
with ProgressBar():
    df.groupby(df.index.dt.to_period('M'))['x'].size().compute()

[########################################] | 100% Completed | 26.86 s
Wall time: 26.9 s


In [11]:
from dask.distributed import Client

In [13]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 15.79 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55713,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.79 GiB
Comm: tcp://127.0.0.1:55738,Total threads: 3
Dashboard: http://127.0.0.1:55744/status,Memory: 3.95 GiB
Nanny: tcp://127.0.0.1:55719,


In [16]:
%%time
df.groupby(df.index.dt.to_period('M'))['x'].size().compute()

Wall time: 17.2 s


timestamp
2000-01    2678400
2000-02    2505600
2000-03    2678400
2000-04    2592000
2000-05    2678400
2000-06    2592000
2000-07    2678400
2000-08    2678400
2000-09    2592000
2000-10    2678400
2000-11    2592000
2000-12    2678400
2001-01    2678400
2001-02    2419200
2001-03    2678400
2001-04    2592000
2001-05    2678400
2001-06    2592000
2001-07    2678400
2001-08    2678400
2001-09    2592000
2001-10    2678400
2001-11    2592000
2001-12    2678400
2002-01    2678400
2002-02    2419200
2002-03    2678400
2002-04    2592000
2002-05    2678400
2002-06    2592000
2002-07    2678400
2002-08    2678400
2002-09    2592000
2002-10    2678400
2002-11    2592000
2002-12    2678400
2003-01    2678400
2003-02    2419200
2003-03    2678400
2003-04    2592000
2003-05    2678400
2003-06    2592000
2003-07    2678400
2003-08    2678400
2003-09    2592000
2003-10     777600
Freq: M, Name: x, dtype: int64

3\. Изучить, как влияет вызов `persist` на производительность

In [18]:
female = {"Edith", "Wendy", "Quinn", "Hannah", "Ursula", "Laura", "Sarah", "Alice"}

In [22]:
female_df = df[df['name'].isin(female)].persist()
female_df

,name,id,x,y,xy
npartitions=1378,,,,,
2000-01-01,object,int32,float64,float64,float64
2000-01-02,...,...,...,...,...
...,...,...,...,...,...
2003-10-09,...,...,...,...,...
2003-10-10,...,...,...,...,...


In [20]:
female_df.groupby('name')['x'].mean().compute()

name
Alice    -0.000330
Edith    -0.000186
Hannah    0.000239
Laura     0.000093
Quinn    -0.000318
Sarah     0.000028
Ursula    0.000438
Wendy    -0.000167
Name: x, dtype: float64

In [25]:
%%time
(female_df['y']*10).mean().compute()

Wall time: 2.31 s


-0.0003847314737873529

## Лабораторная работа 13

__При решении данных задач не подразумевается использования циклов или генераторов Python в ходе работы с пакетами `numpy`, `pandas` и `dask`, если в задании не сказано обратного. Решения задач, в которых для обработки массивов `numpy`, структур `pandas` или структур `dask` используются явные циклы (без согласования с преподавателем), могут быть признаны некорректными и не засчитаны.__

В ходе выполнения все операции вычислений проводятся над `dask.DataFrame` и средствами пакета `dask`, если в задании не сказано обратного. Переход от `dask.DataFrame` к `pd.DataFrame` возможен исключительно для демонстрации результата в конце решения задачи. Если в задаче используются результаты выполнения предыдущих задач, то подразумевается, что вы используете результаты в виде `dask.DataFrame` (то есть то, что было получено до вызова `compute`, а не после).

1\. В архиве `recipes_full.zip` находятся файлы, содержащие информацию об рецептах блюд. Загрузите данные из файлов этого архива в виде `dd.DataFrame` с названием `recipes`. Укажите, что в столбце `submitted` содержатся даты. Выведите на экран информацию о количестве сегментов и типе столбцов. Выведите на экран 5 первых и 5 последних строк таблицы. В случае сообщения об ошибке объясните причину и исправьте ошибку.

In [1]:
import dask.dataframe as dd
import dask

In [2]:
recipes = dd.read_csv('recipes_full_*.csv', dtype={'minutes': 'float64',
       'n_steps': 'float64'})
recipes['submitted']=dd.to_datetime(recipes.submitted)
recipes

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
npartitions=8,,,,,,,,
,int64,object,float64,int64,datetime64[ns],float64,object,int64
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [3]:
recipes.head(5)

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,683970,vant ivoire mickies nothing,33.0,803776,2019-08-22,4.0,pat and gina neely and their family own and op...,9
1,1089012,kremsils mariposa baccala cookies class borage...,23.0,51579,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5
2,1428572,tania lander,0.0,68884,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5
3,1400250,heloise milli asher doogh zojirushi,24.0,678862,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1
4,387709,nutty chocolate chunk cookies,47.0,489552,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10


In [4]:
recipes.tail(5)

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients
278949,1029131,tuti waffle snackies steakhouse,19.0,171345,1973-10-18,4.0,"according to a providence journal article, ama...",4
278950,1700703,noelias cheats crocante fleisch zitumbuwa,1.0,30228,2007-07-01,6.0,if possible sauté the onions and garlic in abo...,1
278951,1910650,rubbed restuffed pelmeni bedouin flavourful,60.0,591905,2009-09-26,3.0,another great recipe to add to the growing swe...,2
278952,713836,stems polpettine peezi,NaN,357389,2003-09-30,4.0,adapted from top secret recipes. love this!,9
278953,660699,clementines,64.0,29196,1973-06-03,6.0,this would make a great start to your holiday ...,8


2\. Выведите на экран следующую информацию:
* количество сегментов в таблице
* объем используемой памяти каждого сегмента
* количество строк в каждом сегменте
* количество строк во всей таблице.

In [5]:
recipes.npartitions

8

In [6]:
recipes.memory_usage_per_partition().compute()

0    17853248
1    17853248
2    17853248
3    17853248
4    17853248
5    17853184
6    17853184
7    17853184
dtype: int64

In [7]:
recipes.map_partitions(lambda x: len(x)).compute()

0    278955
1    278955
2    278955
3    278955
4    278955
5    278954
6    278954
7    278954
dtype: int64

In [8]:
len(recipes)

2231637

3\. В архиве `recipes_additional.zip` находятся файлы, содержащие информацию об рецептах блюд, полученные из другого источника. В связи с этим названия и количество столбцов в этих данных отличаются от того, что находится в архиве `recipes_full.zip`.

Объедините два набора набора данных, согласовав названия столбцов и форматы данных. Добавьте столбец с информацией о том, из какого датасета была получены данные.

In [58]:
recipes_add = dd.read_csv('recipes_additional_*.csv')
recipes_add['recipe_id'] = recipes_add['recipe_id'].map(lambda x: int(x,16),meta=('x', 'int64'))
recipes_add['hours'] = recipes_add['hours'].map(lambda x: x*60,meta=('x', 'float64'))
recipes_add['#tags'] = recipes_add['#tags'].map(lambda x: float(x),meta=('x', 'float64'))
recipes_add['date_of_publication']=dd.to_datetime(recipes_add.date_of_publication)
recipes_add = recipes_add.rename(columns={"#tags": "n_steps", "hours": "minutes", "date_of_publication":"submitted", "recipe_id":"id","#ingredients":"n_ingredients"})  

In [59]:
recipes_add.head()

,name,description,minutes,submitted,n_steps,n_ingredients,id,views_30_days
0,blue cheese tenders,found this recipe in my mom's older recipes. ...,20.0,2008-01-02,12.0,5,337412,2001
1,hindi frappes tikkis siro changling anya jerkll,this is one i found on a post-it in my kitchen...,33.0,1983-06-25,13.0,3,2028894,4691
2,navratan padma beehoon contest carrot,ok...here in the us avocados are used in savor...,26.0,2007-12-19,9.0,5,445682,3823
3,camptown two hurry laddi,i got this out of southern living magazine and...,8.0,1998-06-11,3.0,4,441161,3202
4,down on the farm sausage bake,definite comfort food for me and my family.,75.0,2002-10-16,5.0,13,56611,2143


In [68]:
recipess = dd.multi.concat([recipes,recipes_add])
recipess.head()

,id,name,minutes,contributor_id,submitted,n_steps,description,n_ingredients,views_30_days
0,683970,vant ivoire mickies nothing,33.0,803776.0,2019-08-22,4.0,pat and gina neely and their family own and op...,9,NaN
1,1089012,kremsils mariposa baccala cookies class borage...,23.0,51579.0,2013-03-02,1.0,"a light, tasty and easy to put together chicke...",5,NaN
2,1428572,tania lander,0.0,68884.0,1980-11-09,1.0,a delicious melt in your mouth appetizer. for ...,5,NaN
3,1400250,heloise milli asher doogh zojirushi,24.0,678862.0,2018-04-29,3.0,delicious cream cheese and peach filled cresce...,1,NaN
4,387709,nutty chocolate chunk cookies,47.0,489552.0,2009-08-31,8.0,everyone loves these buttery cookies chock ful...,10,NaN


4\. Выясните, сколько рецептов содержат слово `chocolate`. Выведите на экран набор уникальных идентификаторов авторов таких рецептов.

In [69]:
recipess['description'].str.extract(r'(chocolate)').compute()

ValueError: The columns in the computed data do not match the columns in the provided metadata
Order of columns does not match

5\. При помощи функции `da.histogram` посчитайте значения для построения гистограммы для столбца `n_ingredients`. Визуализируйте полученные результаты при помощи функции `plt.bar`. Добавьте на рисунок 2 вертикальные линии, соответствующие квантилям уровней 0.25 и 0.75. Сделайте масштаб вертикальной оси логарифмическим.

Допускается вычисление статистики по столбцу (применение метода `compute`) до того, как будет вызван `da.histogram`. 

6\. Постройте сводную таблицу при помощи метода `pivot_table`, где по строкам располагаются года, по столбцам - названия месяцев, а в ячейках содержится средняя длина рецептов в данный год и месяц.

7\. Удалите строки, которые содержат пропуске в столбце `contributor_id` и приведите его тип к целочисленному. Для каждого `contributor_id` найдите топ-5 слов, которых этот пользователь наиболее часто использовал в названиях своих рецептов. Выведите на экран статистику для пользователя с ID 1530.

8\. В файлах архива `site_logs.zip` находятся логи системы. Считайте эти логи в виде `dask.bag`, выделите их них дату, время, id службы, уровень лога и сообщение. Объедините дату и время в одну строку. Преобразуйте `dask.bag` в `dask.dataframe`, предварительно оставив только логи уровня "INFO". Подтвердите корректность решения, выведя на экран уникальные значения столбца `level` полученной таблицы.

```
date(ymd)   time(HMS)   service_id level   msg
081110      103041      34         INFO    dfs.FSNamesystem: BLOCK* NameSystem.delete: ...
```

9\. Добавьте в таблицу из задания 8 столбец `is_delete_op`, который содержит флаг: является ли данная операция удалением чего-либо или нет. Для выяснения этого факта проанализизируйте сообщение лога. 

Для каждого лога подсчитайте, сколько из записанных перед ним последних 100 логов являются операциями удаления. Для вычислений используйте метод `map_overlap`. Посчитайте и выведите на экран, сколько существует подпоследовательностей не менее чем из 75 сообщений, содержащих операции удаления.

10\. Сгруппируйте таблицу с логами по часу, в котором был оставлен лог. Под часом имеется в виду час конкретного дня, то связь связка год-месяц-день-час.  Посчитайте, сколько логов было записано за каждый час. Визуализируйте полученную динамику. Добавьте подписи засечек, подписи осей и название рисунка.